# BIXI stations and lanes analysis

In [1]:
import pandas as pd
import geopandas as gpd

import leafmap

In [2]:
# Retrieve bixi stations data
gdf_bixi_stations = gpd.read_file('../data/curated/bixi-stations.json')
gdf_bixi_stations = gdf_bixi_stations[gdf_bixi_stations['WITHIN_CITY_AREA'] == True]
gdf_bixi_stations.head()

,STATIONNAME,STATIONLATITUDE,STATIONLONGITUDE,WITHIN_CITY_AREA,geometry
0,Métro Champ-de-Mars (Viger / Sanguinet),45.510253,-73.556777,True,POINT (-8188302.92 5702211.268)
1,Métro Place-d'Armes (Viger / St-Urbain),45.506314,-73.559671,True,POINT (-8188625.144 5701585.66)
2,Émile-Duployé / Sherbrooke,45.527195,-73.564526,True,POINT (-8189165.524 5704902.986)
3,Marmier / St-Denis,45.531027,-73.598623,True,POINT (-8192961.25 5705511.795)
4,du Parc-Lafontaine / Rachel,45.525512,-73.574245,True,POINT (-8190247.436 5704635.456)


In [3]:
# Load bikelane data
gdf_bikelane = gpd.read_file('../data/raw/bikelane-infra.json')

In [4]:
# Ensure both GeoDataFrames are in the same CRS
gdf_bixi_stations = gdf_bixi_stations.to_crs('EPSG:3857')
gdf_bikelane = gdf_bikelane.to_crs('EPSG:3857')

gdf_bixi_stations.crs, gdf_bikelane.crs

(<Projected CRS: EPSG:3857>
 Name: WGS 84 / Pseudo-Mercator
 Axis Info [cartesian]:
 - X[east]: Easting (metre)
 - Y[north]: Northing (metre)
 Area of Use:
 - name: World between 85.06°S and 85.06°N.
 - bounds: (-180.0, -85.06, 180.0, 85.06)
 Coordinate Operation:
 - name: Popular Visualisation Pseudo-Mercator
 - method: Popular Visualisation Pseudo Mercator
 Datum: World Geodetic System 1984 ensemble
 - Ellipsoid: WGS 84
 - Prime Meridian: Greenwich,
 <Projected CRS: EPSG:3857>
 Name: WGS 84 / Pseudo-Mercator
 Axis Info [cartesian]:
 - X[east]: Easting (metre)
 - Y[north]: Northing (metre)
 Area of Use:
 - name: World between 85.06°S and 85.06°N.
 - bounds: (-180.0, -85.06, 180.0, 85.06)
 Coordinate Operation:
 - name: Popular Visualisation Pseudo-Mercator
 - method: Popular Visualisation Pseudo Mercator
 Datum: World Geodetic System 1984 ensemble
 - Ellipsoid: WGS 84
 - Prime Meridian: Greenwich)

In [5]:
from shapely.ops import nearest_points

# Function to find the nearest bike lane and its distance
def find_nearest_bikelane(station, bikelanes):
    nearest_geom = nearest_points(station.geometry, bikelanes.unary_union)[1]
    distance = station.geometry.distance(nearest_geom)
    return distance

# Apply the function to each station
gdf_bixi_stations['distance'] = gdf_bixi_stations.apply(find_nearest_bikelane, bikelanes=gdf_bikelane, axis=1)

# round distance to nearest meter
gdf_bixi_stations['distance'] = gdf_bixi_stations['distance']

In [6]:
gdf_bixi_stations.sort_values(by='distance', ascending=False, inplace=True)
gdf_bixi_stations.head()

,STATIONNAME,STATIONLATITUDE,STATIONLONGITUDE,WITHIN_CITY_AREA,geometry,distance
728,Brittany / Ainsley,45.525890,-73.650034,True,POINT (-8198684.323 5704695.555),1225.729965
710,de Côme / Jean-Talon,45.588303,-73.577837,True,POINT (-8190647.372 5714618.16),1004.620238
480,des Jockeys / Décarie,45.494044,-73.654495,True,POINT (-8199180.865 5699636.892),919.376639
462,Laird / Glencoe,45.506192,-73.642308,True,POINT (-8197824.227 5701566.143),785.098591
123,de Soissons / de Darlington,45.506899,-73.623526,True,POINT (-8195733.435 5701678.532),748.398384


In [7]:
# Add buffer to the geometry column using the distance
gdf_bixi_stations['geometry'] = gdf_bixi_stations.geometry.buffer(gdf_bixi_stations['distance'])

# Plot stations as a circle with size proportional to the distance
m = leafmap.Map(center=[45.52, -73.6], zoom=13)
m.add_basemap("Google Maps")
m.add_gdf(gdf_bixi_stations, layer_name='BIXI Stations', fill_color='red', fill_opacity=0.5)

m.add_geojson(gdf_bikelane, layer_name='Bike lanes', fill_opacity=0.5, fill_color='green', weight=2)
m


Map(center=[45.52, -73.6], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

In [9]:
# Export geo json
gdf_bixi_stations.to_file('../data/curated/bixi_stations_distance.geojson', driver='GeoJSON')